In [1]:
import os
import re

import django
import fs.path
import numpy as np
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
# from mastspec.views import *
# from mastspec.forms import *
from plotter_utils import modeldict

In [ ]:
seq_id = 10524
color = 'dark green'
m = (MSpec.objects.filter(
    observation__seq_id__iexact='mcam' + str(seq_id)
) & MSpec.objects.filter(
    color__iexact=color
))[0]

In [ ]:
modeldict(m)

In [2]:
for spec in MSpec.objects.all():
    spec.delete()
for obs in MObs.objects.all():
    obs.delete()

In [ ]:
MSpec.objects.all()

In [3]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [4]:
metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in ['sol', 'site', 'drive']:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')

In [ ]:
metaframe

In [5]:
# lots of missing values and no spectra, skip for now
BAD_MCAMS = ['mcam13523']
overlay_images = [
    image for image in input_fs.listdir('DCS_ROI_images')
]
rgb_images = [
    image for image in input_fs.listdir('RGB_images')
]

In [6]:
def get_image_ordinal(mastcam_image_fn_no_ext):
    ordinal_appendage = re.search(r'_[RL](\d)$', mastcam_image_fn_no_ext)
    if ordinal_appendage:
        return ordinal_appendage.group(1)
    else:
        return '1'

In [7]:
for ix, row in metaframe.iterrows():
    # skip observations we think are 'bad'
    if row['seq_id'] in BAD_MCAMS:
        continue
    # drop NaN-valued fields and populate observation SQL fields
    # from CSV fields
    row.dropna(inplace=True)
    obs = MObs(**dict(zip(row.index,row.values)))
    # this is the canonical prefix for image / spectra files 
    obs_identifier = 'sol' + format(
            int(row['sol']), "0>4d"
        ) + '_' + row['seq_id']
    overlay_image_list = [
        image for image in overlay_images 
        if image.startswith(obs_identifier)
    ]
    rgb_image_list = [
        image for image in rgb_images 
        if image.startswith(obs_identifier)
    ]
    # associate observation with images using the convoluted decision tree
    # that appears to have been used to name the images (usually)
    for image in overlay_image_list:
        basename = fs.path.splitext(image)[0]
        if re.search(r'_R\d.*?_ROIs', basename):
            image_eye = 'righteye'
        elif re.search(r'_L\d.*?_ROIs', basename):
            image_eye = 'lefteye'
        # note that we _want_ this to throw a NameError if image_eye is undefined 
        setattr(
            obs, 
            image_eye + '_roi_image_' + get_image_ordinal(basename),
            image
        )
    # note subtle, delicious differences in RGB image naming conventions
    for image in rgb_image_list:
        basename = fs.path.splitext(image)[0]
        if re.search(r'R(_R\d)?$', basename):
            image_eye = 'righteye'
        elif re.search(r'L(_[LR]\d)?$', basename):
            image_eye = 'lefteye'
        setattr(
            obs, 
            image_eye + '_rgb_image_' + get_image_ordinal(basename),
            image
        )
    obs.clean()
    obs.save()
pd.DataFrame(map(modeldict, MObs.objects.all()))

,id,name,spectra_set,observation_ptr,sol,ltst,seq_id,rover_elevation,target_elevation,tau_interpolated,...,righteye_roi_image_8,lefteye_roi_image_8,righteye_rgb_image_1,lefteye_rgb_image_1,righteye_rgb_image_2,lefteye_rgb_image_2,righteye_rgb_image_3,lefteye_rgb_image_3,righteye_rgb_image_4,lefteye_rgb_image_4
0,3372,Goulburn 2x1,plotter.MSpec.None,Observation object (3372),13,13:21:56,mcam00012,-4500.97,-4502.2709,0.725,...,None,None,sol0013_mcam00012_RGB_R_R1.jpg,sol0013_mcam00012_RGB_L_L1.jpg,sol0013_mcam00012_RGB_R_R2.jpg,sol0013_mcam00012_RGB_L_L2.jpg,None,None,None,None
1,3373,Dunes+Mound 1x2,plotter.MSpec.None,Observation object (3373),13,13:30:30,mcam00014,-4500.97,-4496.5050,0.725,...,None,None,sol0013_mcam00014_RGB_R_R1.jpg,sol0013_mcam00014_RGB_L_L1.jpg,sol0013_mcam00014_RGB_R_R2.jpg,sol0013_mcam00014_RGB_L_L2.jpg,None,None,None,None
2,3374,Clast Survey,plotter.MSpec.None,Observation object (3374),24,15:32:24,mcam00119,-4502.61,-4503.1598,0.725,...,None,None,sol0024_mcam00119_RGB_R.jpg,sol0024_mcam00119_RGB_L.jpg,None,None,None,None,None,None
3,3375,Fractures 2x2,plotter.MSpec.None,Observation object (3375),25,12:32:08,mcam00121,-4502.61,-4500.6437,0.725,...,None,None,None,None,None,None,None,None,None,None
4,3376,Hepburn (distant),plotter.MSpec.None,Observation object (3376),25,13:01:29,mcam00126,-4502.61,NaN,0.725,...,None,None,sol0025_mcam00126_RGB_R.jpg,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,4119,Edinburgh Stereo,plotter.MSpec.None,Observation object (4119),2705,None,mcam14177,NaN,NaN,0.349,...,None,None,sol2705_mcam14177_RGB_R.jpg,sol2705_mcam14177_RGB_L.jpg,None,None,None,None,None,None
748,4120,Eshaness Stereo,plotter.MSpec.None,Observation object (4120),2710,None,mcam14191,NaN,NaN,0.349,...,None,None,None,None,None,None,None,None,None,None
749,4121,Edinburgh Stereo,plotter.MSpec.None,Observation object (4121),2712,12:55:20,mcam14203,-4088.69,NaN,0.349,...,None,None,sol2712_mcam14203_RGB_R.jpg,sol2712_mcam14203_RGB_L.jpg,None,None,None,None,None,None
750,4122,Edinburgh Dump Pile,plotter.MSpec.None,Observation object (4122),2726,None,mcam14264,NaN,NaN,0.349,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# for spec in MSpec.objects.all():
#     spec.delete()

In [8]:
SEQ_ID_PATTERN = r"mcam\d+(?=_)"
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('spectra-marslab.csv') and file.startswith('sol'))
]
for spec_file in spec_files:
    seq_id = re.search(SEQ_ID_PATTERN, spec_file).group()
    try:
        observation = MObs.objects.get(seq_id__iexact=seq_id)
    except ObjectDoesNotExist:
        print("no observation for " + spec_file, seq_id)
        continue
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    image_number = get_image_ordinal(spec_file)
    for _, row in frame.iterrows():
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in MSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # we would like these metadata to be carried on the parent
        # observation rather than on the spectrum (i.e., we don't need
        # an extra pivot because we already have a FOREIGN KEY.) but
        # we want to make sure they match!
        assert row['sol'] == observation.sol
        assert row['seq_id'] == observation.seq_id
        row.drop(['sol','seq_id', 'instrument'], inplace=True)
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        metadata = dict(row) | {
            'observation': observation,
            'image_number': image_number,
            'filename': spec_file
        }
        spectrum = MSpec(**metadata)
        spectrum.clean()
        spectrum.save()


no observation for sol0614_mcam02954_spectra-marslab.csv mcam02954
no observation for sol1333_mcam06935_spectra-marslab.csv mcam06935


In [ ]:
modeldict

In [ ]:
MSpec.objects.filter(observation__sol__iexact=232)[0].image_files()

In [ ]:
image_files(modeldict(
    MSpec.objects.filter(observation__sol__iexact=232)[0]
))

In [ ]:
modeldict(
    MObs.objects.get(seq_id__iexact='mcam01099')
)

In [ ]:
for obs in MObs.objects.all():
    if 'sol0232_mcam01099_L10G6_ROIsOVERLAYjpg_browse.jpg' in modeldict(obs).values():
        break

In [ ]:
modeldict(obs)